In [ ]:
pip install scikit-surprise
pip install pandas

In [ ]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load the dataset
def load_data():
    # Load a dataset (you can replace this with your own dataset)
    # Assuming a CSV file with columns: 'userId', 'movieId', 'rating'
    df = pd.read_csv('ratings.csv')  # Use your own ratings dataset
    
    return df

# Prepare the data for the Surprise library
def prepare_data(df):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    return data

# Train-test split
def train_test_data(data):
    trainset, testset = train_test_split(data, test_size=0.25)
    return trainset, testset

# Train the collaborative filtering model using KNNBasic algorithm
def train_model(trainset):
    # We'll use user-based collaborative filtering here
    algo = KNNBasic(k=40, sim_options={'name': 'cosine', 'user_based': True})
    
    algo.fit(trainset)
    return algo

# Make predictions on the test set
def evaluate_model(algo, testset):
    predictions = algo.test(testset)
    accuracy.rmse(predictions)

# Recommend movies for a specific user
def get_movie_recommendations(algo, user_id, df, top_n=10):
    # Get the list of all movie ids
    all_movie_ids = df['movieId'].unique()
    
    # Get the movies already rated by the user
    user_rated_movies = df[df['userId'] == user_id]['movieId']
    
    # Filter out the movies the user has already rated
    unrated_movies = [movie_id for movie_id in all_movie_ids if movie_id not in user_rated_movies.values]
    
    # Predict ratings for the unrated movies
    predictions = []
    for movie_id in unrated_movies:
        pred = algo.predict(user_id, movie_id)
        predictions.append((movie_id, pred.est))
    
    # Sort by predicted rating in descending order and return top N movies
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    # Extract movie ids of top N movies
    top_movie_ids = [movie_id for movie_id, rating in predictions[:top_n]]
    
    return top_movie_ids

# Main function to execute the recommendation system
def main():
    # Load dataset
    df = load_data()
    
    # Prepare data
    data = prepare_data(df)
    
    # Train-test split
    trainset, testset = train_test_data(data)
    
    # Train model
    algo = train_model(trainset)
    
    # Evaluate model
    evaluate_model(algo, testset)
    
    # Recommend movies for a specific user (for example, user with ID 1)
    user_id = 1
    recommended_movies = get_movie_recommendations(algo, user_id, df, top_n=10)
    
    print(f"Recommended movies for user {user_id}: {recommended_movies}")

if __name__ == '__main__':
    main()
